# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [2]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [4]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 8763ec5b


In [5]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Quick test
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** are sophisticated AI agents designed to handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agent Architecture

```
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning as **context engineering**:
- Todo lists provide **persistent context** about what needs to be done
- File systems serve as **extended memory** beyond the context window
- Subagents enable **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [6]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    # Update the index of the todo list so the output is continuous
    start = len(TODO_STORE)
    for i, todo in enumerate(todos):
        todo_id = f"todo_{start + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [7]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_2, todo_3

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_2] Research sleep improvement strategies (pending)
⬜ [todo_3] Create personalized sleep plan (pending)


In [8]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
🔄 [todo_2] Research sleep improvement strategies (in_progress)
⬜ [todo_3] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [9]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/yingzheli/aimakerspace/AIE9/07_Deep_Agents/workspace


In [10]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] comprehensive_morning_routine_guide.md (23340 bytes)
[FILE] morning_routine_guide.md (49920 bytes)
[FILE] personalized_sleep_improvement_plan.md (6320 bytes)
[DIR] research
[FILE] sleep_improvement_research.md (13356 bytes)


In [11]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] comprehensive_sleep_guide.md (19369 bytes)
[FILE] sleep_notes.md (242 bytes)


In [12]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [13]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/yingzheli/aimakerspace/AIE9/07_Deep_Agents/workspace


In [49]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Sleep Improvement Plan is Complete! 

I've created a comprehensive 8-week sleep transformation plan specifically tailored to address your three main challenges:

### **Key Highlights of Your Plan:**

🎯 **Phase-Based Approach**: 
- **Weeks 1-2**: Foundation building (fixed wake time + digital sunset)
- **Weeks 3-4**: Gradual bedtime consolidation  
- **Weeks 5-8**: Quality enhancement and morning fatigue solutions

📱 **Phone/Screen Solutions**:
- Complete "digital sunset" 1 hour before bed
- Phone charging outside bedroom
- Alternative relaxing activities provided

⏰ **Schedule Consistency**:
- Fixed wake time (most critical element)
- Gradual bedtime adjustment (15 min every 3-4 days)
- Weekend guidelines to maintain progress

🌅 **Morning Fatigue Solutions**:
- Light therapy recommendations
- Sleep window optimization
- Temperature and routine adjustments

### **The plan includes:**
- ✅ Week-by-week implementation guide
- ✅ Troubleshooting strategie

In [50]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_2] Research evidence-based sleep improvement strategies (completed)
✅ [todo_3] Create personalized sleep improvement plan (completed)
✅ [todo_4] Save plan to file (completed)
✅ [todo_5] Research sleep hygiene best practices (completed)
✅ [todo_6] Research consistent sleep schedule strategies (completed)
✅ [todo_7] Research screen time impact and reduction strategies (completed)
✅ [todo_8] Research morning fatigue causes and solutions (completed)
✅ [todo_9] Compile scientific backing and studies (completed)
✅ [todo_10] Create structured research summary (completed)


Workspace contents:
  [FILE] comprehensive_morning_routine_guide.md (23340 bytes)
  [FILE] morning_routine_guide.md (49920 bytes)
  [FILE] personalized_sleep_improvement_plan.md (6320 bytes)
  [DIR] research/
  [FILE] sleep_improvement_research.md (13356 bytes)


---
## Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:

Overall the effectiveness of using todo lists for planning depends on the complexity of the task.

- For simple tasks that do not require much planning and long running tasks (e.g., the wellness planner we built in previous classes), todo overhead can slow things down quite a bit. For example, the extra LLM calls for planning/tracking add latency. 
- For complext tasks that could take long time to run and complete (e.g., deep research into a topic), todos planning is lot more beneficial since the todo lists enables agents to break down complex task into granular task, allow subagents spawning to complete the tasks, track and resume the progress.

Regarding granularity of the tasks:
- Too granular todo will cause context to fill up quickly with AI/human messages and tool call results, also the overall task may be taking longer and more costly to complete;
- By contrast, very coarse todos may cause each todo itself to be a complext task, which may cause loss of trackability for the task or less desirable output.
- Sweet spot maybe for each todo to represent a meaningful intermeidate deliverable sub-task (e.g., research a particular topic, write/update a file, etc)

How to handle incomplete todos:
- Add a max iteration limits to break the loop and return results to the user to avoid infinite loop
- Save the todos progress to file(s) so the work isn't lost and can be restored
- Let user attempt to resume unfinished tasks, e.g., if they add a prompt to "continue where you left off".



## Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:

Here are some considerations on how would I design the context management strategy for this wellness agent:

For the large health document, there's no need to inject the whole document into context, which will fill up a significant chunk of our context window and causing context rot or high serving cost. Instead, we can chunk the document store in a vector database, and retrieve only relevant chunks when the user asks about a specific topic. This can be done either using a traditional RAG approach (e.g., build a tool for LLM) or use the semantic long term memory approach (e.g., retrieve the relevant text and load them into the system prompt).

For tracking user metrics over time, we can also either store the metrics in a file in the file system, or in a long term memory store which will be retrieved and loaded into the context (e.g., like what we implemented in the long term memory class). However, as user's metrics grow and the amount of data increases, a hybrid approach is likely better. For example, use the file system to store all raw user metrics (e.g., for the past year's data); while using the long term memory to store only the most recent metrics (e.g., past week or month's data). In this way, the user's recent metrics will always be loaded from the long term memory into the prompt, so they can ask for the recent trends (e.g., past week's trends). If they want to find historical data from long time ago, we can use tool calling to query a filesystem tool to grep the data from the file (e.g., metric from 6 months ago).

For cricital user conditions (allergies, medications), these are best suited to be stored in a long-term store such as within user profile, and they will always be loaded into the prompt without any tool calling, since we do not want to risk agent to give dangerous unsafe answers to the user due to missed tool calling (which is not always guaranteed).

So to summarize:

Context that can go into files, which agent can use tool calling to retrieve dynamically when needed:
- Large knowledge base documents (e.g., health guides, research reports)
- Raw historical metrics
- Agent generated plans, reports, notes
- Agent summarized old conversation history

Context that can be stored in long-term store which will always be injected into prompt and not offloaded to tool calling:
- Critical user profile: allergies, medications, medical conditions 
- User's preferences and other constraints
- Recent user metrics summary
- Other information that may cause safety/compliance/security issues if forgotten

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [19]:
### YOUR CODE HERE ###

from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide

# Add a tool to read from the data folder
@tool
def read_wellness_guide() -> str:
    """Read the Health Wellness Guide reference document.
    
    Returns:
        The full contents of the HealthWellnessGuide.txt file
    """
    path = Path("data/HealthWellnessGuide.txt")

    if not path.exists():
        return f"File not found: {path}"
    
    return path.read_text()

# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
    read_wellness_guide,
]
# Step 3: Create the agent with a research-focused system prompt

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)


# Create a basic Deep Agent
wellness_research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-5-20250929"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where research files are stored
    system_prompt="""You are a Wellness Research Agent that produces structured reports.

When given a research task:
1. Create a todo list to track your research process
2. Use your available tools to extract fact-based and evidence-based information
3. Extract and synthesize relevant information
4. Save your findings to a structured markdown file
5. Update todo status as you complete each step

Be thorough but concise. Always explain your reasoning. 
Focus on evidence-based information and cite the guide where relevant."""
)

print(f"Basic Research Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

# Step 4: Test with the stress management research task
TODO_STORE.clear()

final_response = None

for chunk in wellness_research_agent.stream(
    {"messages": [{
        "role": "user",
        "content": "Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."
    }]},
    stream_mode="updates"
):
    for node, values in chunk.items():
         # Skip middleware nodes
        if "Middleware" in node:
            continue
        
        print(f"\n[Node: {node}]")

        if "messages" in values:
            for msg in values["messages"]:
                if hasattr(msg, 'content') and msg.content:
                    print(msg.content[0:200])
                    final_response = msg

Basic Research Agent created!
File operations sandboxed to: /Users/yingzheli/aimakerspace/AIE9/07_Deep_Agents/workspace

[Node: model]
[{'text': "I'll help you research stress management techniques and create a comprehensive guide. Let me start by organizing this task and accessing the reference material.", 'type': 'text'}, {'id': 'toolu_01Gyuxyj5Mhc8bK2nXLpWVTB', 'input': {'todos': [{'title': 'Read the Health Wellness Guide reference document', 'description': 'Extract information about stress management techniques from the wellness guide'}, {'title': 'Identify and document at least 5 evidence-based stress management strategies', 'description': 'Synthesize information from the guide into clear, actionable strategies'}, {'title': 'Create a comprehensive markdown guide', 'description': 'Structure the findings into a well-organized document with sections for each strategy'}, {'title': 'Review and finalize the guide', 'description': 'Ensure all strategies are evidence-based and clearly exp

In [20]:

# Check final task list and workspace contents
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Read the Health Wellness Guide reference document (completed)
✅ [todo_2] Identify and document at least 5 evidence-based stress management strategies (completed)
✅ [todo_3] Create a comprehensive markdown guide (completed)
✅ [todo_4] Review and finalize the guide (completed)


Workspace contents:
  [FILE] comprehensive_morning_routine_guide.md (23340 bytes)
  [FILE] comprehensive_stress_management_guide.md (13753 bytes)
  [FILE] evidence_based_stress_management_guide.md (32324 bytes)
  [FILE] morning_routine_guide.md (49920 bytes)
  [FILE] personalized_sleep_improvement_plan.md (6320 bytes)
  [DIR] research/
  [FILE] sleep_improvement_research.md (13356 bytes)
  [FILE] stress_management_guide.md (19086 bytes)


In [22]:
print("Final response:")
print(final_response.content)

Final response:
Perfect! I've successfully completed your research task on stress management techniques. Here's a summary of what I've created:

## Comprehensive Stress Management Guide Created ✅

I've researched and compiled a **comprehensive, evidence-based guide** with **7 major stress management strategies** (exceeding your requirement of 5), saved to `/evidence_based_stress_management_guide.md`.

### The 7 Evidence-Based Strategies Include:

1. **Deep Breathing Techniques** - Box breathing method for immediate stress relief
2. **Progressive Muscle Relaxation** - Systematic tension/release for physical stress
3. **Mindfulness and Meditation** - 5 different meditation types with practice guidelines
4. **Physical Exercise** - Aerobic, mind-body, and strength training options
5. **Sleep Hygiene Optimization** - Comprehensive sleep practices for stress resilience
6. **Grounding Techniques** - The 5-4-3-2-1 method and other present-moment practices
7. **Lifestyle Management** - Time man

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [23]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [24]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [25]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created a comprehensive morning routine guide for better energy. Here's what I delivered:

## ✅ **Project Complete!**

**📄 File Created:** `/ultimate_morning_routine_guide.md`

### **What's Included:**

1. **🔬 Science-Based Research:**
   - Circadian rhythm biology and how morning activities affect energy
   - Exercise timing research and metabolic benefits
   - Nutritional science around breakfast vs. intermittent fasting
   - Sleep science and stress hormone regulation
   - Hydration research and cognitive impacts

2. **📋 Practical Implementation:**
   - **3 Complete Routine Templates:** 15-minute, 30-minute, and 60-minute versions
   - **4 Core Pillars:** Light/Environment, Movement/Exercise, Hydration/Nutrition, Mindset/Mental Preparation
   - **Lifestyle Customizations:** For busy parents, shift workers, non-morning people, travelers, and advanced practitioners

3. **🛠 Real-World Solutions:**
   - **6 Common mistakes** and how to av

In [26]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines (completed)
✅ [todo_2] Create comprehensive morning routine guide (completed)
✅ [todo_3] Save guide as formatted markdown file (completed)

Generated files in workspace:
  [FILE] comprehensive_morning_routine_guide.md (23340 bytes)
  [FILE] comprehensive_stress_management_guide.md (13753 bytes)
  [FILE] evidence_based_stress_management_guide.md (32324 bytes)
  [FILE] morning_routine_guide.md (49920 bytes)
  [FILE] personalized_sleep_improvement_plan.md (6320 bytes)
  [DIR] research/
  [FILE] sleep_improvement_research.md (13356 bytes)
  [FILE] stress_management_guide.md (19086 bytes)
  [FILE] ultimate_morning_energy_guide.md (29599 bytes)
  [FILE] ultimate_morning_routine_guide.md (19605 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [27]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [28]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [29]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [30]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! Great to connect with you again. I see from your profile that your primary goal is to improve energy levels with better sleep as a secondary goal. I also note you prefer morning exercise and have mild anxiety, so I'll tailor my recommendations accordingly.

## Recommended Exercise Routine for You

Given your goals and preferences, here's a morning routine designed to boost energy levels and promote better sleep:

### **Weekly Schedule (4-5 days/week)**

**Monday, Wednesday, Friday - Energy Boosting Circuit (20-25 minutes)**
- 5-minute gentle dynamic warm-up (arm circles, leg swings, light marching)
- 15-minute circuit (3 rounds, 45 seconds work, 15 seconds rest):
  - Bodyweight squats
  - Modified push-ups (knee or wall variations)
  - Walking lunges
  - Plank hold
  - Jumping jacks (or step-touches for lower impact)
- 5-minute cool-down with gentle stretching

**Tuesday, Thursday - Mindful Movement (15-20 minutes)**
- Yoga flow or tai chi-style movements
- Foc

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. **Instead of loading all tools upfront, agents can load specialized capabilities on demand.**

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [31]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [32]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [33]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [34]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [35]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Wellness Assessment Summary

I've completed your comprehensive wellness assessment and saved the full report to your workspace. Here are the key findings and recommendations:

### **Your Current Strengths:**
- No major health conditions give you a great foundation
- Your vegetarian diet is already health-conscious 
- You have clear, achievable goals
- At 35, your body is still very responsive to positive changes

### **Priority Areas for Improvement:**
1. **Movement Integration** (Top Priority) - Your sedentary work life is likely the biggest factor affecting both your weight and sleep
2. **Sleep Hygiene** - Poor sleep is sabotaging your metabolism and energy
3. **Nutritional Optimization** - Fine-tuning your vegetarian diet for weight loss

### **Start Today - 3 Immediate Actions:**
1. Set hourly movement reminders on your phone/computer
2. Create a 30-minute screen-free wind-down routine before bed
3. Track your water intake - aim for 8 glasses daily

### **Yo

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [36]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents 0.0.17


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [37]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [38]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [39]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! I've successfully created your comprehensive 2-week wellness program, Alex! 🎉

## What I've Created for You:

### 📁 **4 Complete Reference Files:**

1. **alex_complete_wellness_program.md** - Your master program integrating everything
2. **alex_exercise_plan.md** - Complete 6-session workout plan (3x/week, 30 min each)
3. **alex_nutrition_plan.md** - 14-day vegetarian meal plan with shopping lists
4. **alex_stress_sleep_plan.md** - Daily stress management and sleep optimization techniques

### 🎯 **Program Highlights Tailored for You:**

**Exercise Plan:**
- 6 total sessions over 2 weeks (Mon/Wed/Fri)
- Morning-focused to boost your energy levels
- Progressive difficulty from Week 1 to Week 2
- Anxiety-reducing cool-downs with deep breathing

**Nutrition Plan:**
- 100% vegetarian with complete proteins
- Energy-optimizing meals timed every 3-4 hours
- Anxiety-supporting foods (magnesium, omega-3s)
- Detailed prep times and shopping lists for both weeks


In [40]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Initial Assessment (completed)
✅ [todo_2] Create Exercise Plan (completed)
✅ [todo_3] Develop Nutrition Plan (completed)
✅ [todo_4] Design Stress & Sleep Program (completed)
✅ [todo_5] Create Master Program Document (completed)
✅ [todo_6] Save Individual Plan Files (completed)

GENERATED FILES
  [FILE] alex_complete_wellness_program.md (7605 bytes)
  [FILE] alex_exercise_plan.md (4625 bytes)
  [FILE] alex_nutrition_plan.md (7785 bytes)
  [FILE] alex_stress_sleep_plan.md (12019 bytes)
  [FILE] comprehensive_morning_routine_guide.md (23340 bytes)
  [FILE] comprehensive_stress_management_guide.md (13753 bytes)
  [FILE] evidence_based_stress_management_guide.md (32324 bytes)
  [DIR] exercise_programs/
  [FILE] meal_plan_14_days_vegetarian.txt (6677 bytes)
  [FILE] morning_routine_guide.md (49920 bytes)
  [FILE] personalized_sleep_improvement_plan.md (6320 bytes)
  [DIR] research/
  [FILE] sleep_improvement_research.md (13356 bytes)
  [FILE] stress_management_gu

In [41]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of alex_complete_wellness_program.md:
# Alex's Complete 2-Week Wellness Program

## Program Overview

Welcome to your personalized 2-week wellness transformation! This comprehensive program addresses your three key goals:

🏋️ **Building a consistent exercise routine** (3x/week, 30 minutes)  
🥗 **Improving your vegetarian diet** for sustained energy  
🧘 **Managing work stress and improving sleep** quality

Based on your profile, this program specifically supports:
- **Primary Goal**: Improving energy levels throughout the day
- **Secondary Goal**: Better sleep quality and duration
- **Special Considerations**: Vegetarian diet, mild anxiety management

---

## Your Daily Wellness Schedule

### **Morning Routine (5-15 minutes)**
1. **Hydration**: Start with 1-2 glasses of water
2. **Mindfulness Practice**: 5-10 minutes from your stress management plan
3. **Movement**: Exercise days (Mon/Wed/Fri) or gentle stretching
4. **Nutrition**: Balanced breakfast from your meal plan

### *

---
## Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
- When should subagents share tools vs have distinct tools?
  - Shared tools make sense for common agent operations such as file system actions (which they can inherit file backend from main agents), tools to track/update todos (each sub-agent use the tool to track its own todos); web search; tools to load long term memory such as user-profile
  - Distinct tools make sense for domain specific actions (e.g., nutrition specialist only search nutrition knowledge base); or when the tool has safety or cost implications, for example tools that use expensive APIs or sensitive data should be restricted only to the subagent that needs them.

- How do you decide which model to use for each subagent?
  - Depending on the task difficulty, we can choose the cheapest and faster model that gets the subagent's job done. For example, we can use the cheaper and faster models for straightforward tasks like summarization, data extraction, or formatting. An then use more capable models for tasks requiring more reasoning, high-quality writing, or safety-critical decisions.


- Sub-agents need to specialization needs to be broad enough to be useful, and narrow enough to provide a specialized task

## Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
*Your answer here*

- Safety guardrails so that we need to respect user privacy and disclaimers 
- Regulation and compliance check
- Backup and restore is needed for prod


---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [ ]:
### YOUR CODE HERE ###

# Step 1: Define your subagent configurations


# Step 2: Create any additional tools you need


# Step 3: Build the main coordinator agent


# Step 4: Test with a user creating their 30-day challenge


# Step 5: Simulate a daily check-in and adaptation


---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - They can manage long-horizon, multi-step tasks effectively
2. **Planning is context engineering** - Todo lists and files serve as extended memory for the agent
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)